# Data Scientist, Science and Natural Perlis Interview code by Isabel Smith 

In [ ]:
import pandas as pd
import folium
import numpy as np
from folium.plugins import HeatMap
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
import matplotlib.colors as mcolors
import matplotlib.dates as mdates
import numpy as np
import pandas as pd
from scipy import stats

# Table of Contents

1. [Importing Data and Initial Analysis](#Importing-Data-and-Initial-Analysis)
   1.1. [EM-DAT Data](#EM-DAT-Data)  
   1.1.1 [Percentage of events per decade](#Percentage-of-events-per-decade)
   1.1.2 [Exploring California](#Exploring-California)
   1.2. [USGS Data](#USGS-Data)
   1.2.1 [Percentage of Events](#Percentage-of-Events)

2. [Comparison Interactive Figure](#Comparison-Interactive-Figure)

3. [Exploring USGS Data Fully](#Exploring-USGS-Data-Fully)

4. [Exploring EM-DAT Fully](#Exploring-EM-DAT-Fully)
   4.1. [Variables in EM-DAT Over Time](#Variables-in-EM-DAT-Over-Time)
   4.2. [Re-insurance Metrics](#Re-insurance-Metrics)

5. [Comparison with Other Variables](#Comparison-with-Other-Variables)



# 1.0 Importing Data plus inital analysis 


## 1.1 EM-DAT Data

The EM-DAT dataset contains information on disasters worldwide. In this section, we will load the dataset and perform some initial exploratory data analysis (EDA).

In [ ]:
# Upload the dataset
df = pd.read_excel("/Users/isabelsmith/Documents/axa/EMDAT_Geophysical_Hydrological_Meteorological_North_American_Hazards_1900_2024[56].xlsx")
df_Geo = pd.DataFrame(df)

# Filter earthquakes in the USA with magnitude >= 5
df_quakes = df_Geo[(df_Geo['Disaster Type'] == 'Earthquake') &
                   (df_Geo['Country'] == 'United States of America') &
                   (df_Geo['Magnitude'] >= 5)]

# Filter based on coordinates for Conterminous USA
lat_min, lat_max = 24.6, 50.0
lon_min, lon_max = -125.0, -65

# Filter rows where Latitude and Longitude are within bounds
df_quakes_Mex_included = df_quakes[(df_quakes['Latitude'] >= lat_min) & 
                                   (df_quakes['Latitude'] <= lat_max) &
                                   (df_quakes['Longitude'] >= lon_min) & 
                                   (df_quakes['Longitude'] <= lon_max)]

# Remove Mexico earthquake
df_quakes = df_quakes_Mex_included[df_quakes_Mex_included['DisNo.'] != '1915-0004-USA']

# Multiply the relevant columns by 1000 to convert '000 US$' to US$
df_quakes['Insured Damage, Adjusted'] *= 1000
df_quakes['Total Damage, Adjusted'] *= 1000

## 1.1.1 Percentage of events per decade

In [ ]:
df_quakes['Year'] = df_quakes['DisNo.'].str.split('-').str[0].astype(int)
df_quakes['Decade'] = (df_quakes['Year'] // 10) * 10
decade_counts = df_quakes['Decade'].value_counts().sort_index()

# Step 1: Calculate percentage of events per decade
total_events = len(df_quakes)
decade_percentages = (decade_counts / total_events) * 100

# Step 2: Combine the counts and percentages into a single DataFrame
decade_summary = pd.DataFrame({
    'Event Count': decade_counts,
    'Percentage of Total Events': decade_percentages
})
decade_summary 

## 1.1.2 Exploring Califonia

In [ ]:
# Defining the latitude and longitude boundaries for California (sourced from https://www.mapsofworld.com/usa/states/california/lat-long.html)
lat_min, lat_max = 32.5, 42.0
lon_min, lon_max = -124.5, -113.5

# Filtering for earthquakes that occurred in California
california_quakes = df_quakes[(df_quakes['Latitude'] >= lat_min) & 
                              (df_quakes['Latitude'] <= lat_max) & 
                              (df_quakes['Longitude'] >= lon_min) & 
                              (df_quakes['Longitude'] <= lon_max)]

# Counting the number of earthquakes in California
num_events_california = california_quakes.shape[0]

# Counting the total number of earthquakes in the entire dataset
total_events = df_quakes.shape[0]

# Calculate the percentage of events in California
percentage_california = (num_events_california / total_events) * 100

print(f"Number of earthquake events in California: {num_events_california}")
print(f"Percentage of earthquake events in California: {percentage_california:.2f}%")


## 1.2 USGS Data

The USGS dataset contains geological data related to earthquakes and other natural hazards. We will load and analyze this data as well.


In [ ]:
def process_and_filter_emdat_geophysical_data(file_path):
    """
    Uploading the geophysical hazards dataset. Despite downloading the contiguous US region, 
    several points developed over Mexico and Canada. This function removes them to focus on the US.

    Parameters:
        file_path (str): Path to the input Excel file.

    Returns:
        DataFrame: Filtered DataFrame containing cleaned geophysical data points.
    """
    # Load dataset
    df_data = pd.read_excel(file_path)
    df = pd.DataFrame(df_data)
    
    # Reverse the dataset for easier comparison, swapping to start at 1900
    df = df.iloc[::-1].reset_index(drop=True)

    # Part 1: Filter to exclude points outside the contiguous US
    def filter_us_points(df, lat_min=24.6, lat_max=50.0, lon_min=-125.0, lon_max=-65.0):
        # Bounding box filter
        in_box = (
            (df['Latitude'] >= lat_min) & (df['Latitude'] <= lat_max) &
            (df['Longitude'] >= lon_min) & (df['Longitude'] <= lon_max)
        )
        
        # Exclude Canada and Mexico
        not_canada = df['Latitude'] < 49.0  # Approximation for Canada
        not_mexico = ~((df['Latitude'] < 32.0) & (df['Longitude'] > -117.0))  # Approximation for Mexico
        
        # Combine filters
        is_us = in_box & not_canada & not_mexico
        
        # Return filtered DataFrame
        return df[is_us]

    df = filter_us_points(df)

    # Part 2: Further removal of points outside specific excluded regions
    exclusions = [
        # 1. Ottawa-Sudbury-Peterborough region
        (44.0, 47.5, -83.5, -75.0),
        # 2. South of Tijuana to San Felipe
        (30.0, 32.5, -116.5, -114.0),
        # 3. West into the Pacific past Guadalupe
        (28.0, 33.0, -120.0, -116.5),
        # 4. East to Puerto Peñasco
        (30.5, 32.5, -114.5, -112.0),
        # 5. West of Quebec
        (46, 49, -72, -30.0)
    ]

    for lat_min, lat_max, lon_min, lon_max in exclusions:
        df = df[
            ~(
                (df['Latitude'] >= lat_min) & (df['Latitude'] <= lat_max) &
                (df['Longitude'] >= lon_min) & (df['Longitude'] <= lon_max)
            )
        ]

    # Part 3: Remove rows with specific 'DisNo.' values
    times_to_exclude = [
        '1979-10-15T23:16:53.910Z',
        '1951-01-24T07:16:52.620Z',
        '1976-05-16T08:35:14.800Z'  # Added this time
    ]
    df = df[~df['time'].isin(times_to_exclude)]

    return df

file_path = "/Users/isabelsmith/Documents/axa/Axa_part2_data.xlsx"
df_USGS = process_and_filter_emdat_geophysical_data(file_path)

## 1.2.1 Percentage of events

In [ ]:
df_USGS['Year'] = df_USGS['time'].str.split('-').str[0].astype(int)
df_USGS['Decade'] = (df_USGS['Year'] // 10) * 10
decade_counts = df_USGS['Decade'].value_counts().sort_index()

# Step 1: Calculate percentage of events per decade
total_events = len(df_USGS)
decade_percentages = (decade_counts / total_events) * 100

# Step 2: Combine the counts and percentages into a single DataFrame
decade_summary_USGS = pd.DataFrame({
    'Event Count': decade_counts,
    'Percentage of Total Events': decade_percentages
})
decade_summary_USGS 

# 2.0 Comparison interactive figure

In [ ]:
def create_combined_earthquake_map_with_time_and_magnitude(df_quakes, df_USGS):
    """
    Here I have created an interactive map plotting earthquakes from df_quakes in blue, df_USGS in red,
    and overlapping points in black. Displays 'Time' and 'Magnitude' once clicked on.

    Parameters:
        df_quakes (DataFrame): DataFrame containing earthquake data with 'Latitude', 'Longitude',
                               'mag' (magnitude), and 'DisNo.' (time).
        df_USGS (DataFrame): DataFrame containing earthquake data with 'Latitude', 'Longitude',
                             'Magnitude', and 'Time'.

    Returns:
        df_sub_1 (DataFrame): Rows from df_USGS that overlap with df_quakes.
        df_sub_2 (DataFrame): Rows from df_quakes that overlap with df_USGS.
    """
    # Extract latitude, longitude, magnitude, and time from both DataFrames
    lat_quakes = df_quakes['Latitude']
    lon_quakes = df_quakes['Longitude']
    mag_quakes = df_quakes['Magnitude']  # Magnitude in df_quakes
    time_quakes = df_quakes['DisNo.']  # Time in df_quakes

    lat_usgs = df_USGS['Latitude']
    lon_usgs = df_USGS['Longitude']
    mag_usgs = df_USGS['mag']
    time_usgs = df_USGS['time']

    # Define a function to check proximity
    def is_close(lat1, lon1, lat2, lon2, tolerance=0.01):
        return abs(lat1 - lat2) <= tolerance and abs(lon1 - lon2) <= tolerance

    # Find overlapping points
    overlap_indices_usgs = []
    overlap_indices_quakes = []
    for i, (lat_usgs_val, lon_usgs_val) in enumerate(zip(lat_usgs, lon_usgs)):
        for j, (lat_quakes_val, lon_quakes_val) in enumerate(zip(lat_quakes, lon_quakes)):
            if is_close(lat_usgs_val, lon_usgs_val, lat_quakes_val, lon_quakes_val):
                overlap_indices_usgs.append(i)  # USGS index
                overlap_indices_quakes.append(j)  # Quakes index
                break  # Stop once an overlap is found

    # Create subsets for overlapping points
    df_sub_1 = df_USGS.iloc[overlap_indices_usgs].reset_index(drop=True)
    df_sub_2 = df_quakes.iloc[overlap_indices_quakes].reset_index(drop=True)

    # Initialize the map at the center of the two datasets
    center_lat = (lat_quakes.mean() + lat_usgs.mean()) / 2
    center_lon = (lon_quakes.mean() + lon_usgs.mean()) / 2
    m = folium.Map(location=[center_lat, center_lon], zoom_start=5)

    # Add points from df_quakes in blue with time and magnitude
    for lat, lon, mag, time in zip(lat_quakes, lon_quakes, mag_quakes, time_quakes):
        tooltip_content = f"<strong>Time (DisNo.):</strong> {time}<br><strong>Magnitude:</strong> {mag}"
        folium.CircleMarker(
            location=[lat, lon],
            radius=4,
            color='blue',
            fill=True,
            fill_opacity=0.6,
            tooltip=tooltip_content
        ).add_to(m)

    # Add points from df_USGS in red with time and magnitude
    for lat, lon, mag, time in zip(lat_usgs, lon_usgs, mag_usgs, time_usgs):
        tooltip_content = f"<strong>Time:</strong> {time}<br><strong>Magnitude:</strong> {mag}"
        folium.CircleMarker(
            location=[lat, lon],
            radius=4,
            color='red',
            fill=True,
            fill_opacity=0.6,
            tooltip=tooltip_content
        ).add_to(m)

    # Add overlapping points in black with time and magnitude
    for lat, lon, mag, time in zip(df_sub_1['Latitude'], df_sub_1['Longitude'], df_sub_1['mag'], df_sub_1['time']):
        tooltip_content = f"<strong>Time:</strong> {time}<br><strong>Magnitude:</strong> {mag}"
        folium.CircleMarker(
            location=[lat, lon],
            radius=6,  # Slightly larger radius to highlight
            color='black',
            fill=True,
            fill_opacity=1.0,
            tooltip=tooltip_content
        ).add_to(m)

    # Save the map
    m.save('earthquake_map_with_time_and_magnitude.html')

    return df_sub_1, df_sub_2

df_sub_1, df_sub_2 = create_combined_earthquake_map_with_time_and_magnitude(df_quakes, df_USGS)

# 3.0 Exploring USGS Data fully 

In [ ]:
def plot_earthquake_data_split_filled_scaled_combined(df_USGS):
    """
    Creating two plots to explore USGS over time. 
    - The first plot shows 'mag', 'not', and 'dmin'.
    - The second plot shows 'rms', 'depth', and 'gap'.
    
    Parameters:
        df_USGS (DataFrame): DataFrame with earthquake data containing 
                             'time', 'mag', 'not', 'dmin', 'rms', 'depth', and 'gap'.
    """
    # Ensure the Date column exists and is properly formatted
    if 'Date' not in df_USGS.columns:
        df_USGS['Date'] = pd.to_datetime(df_USGS['time'])  # Convert 'time' column to datetime

    # Fill missing values in numeric columns
    numeric_cols = df_USGS.select_dtypes(include=[np.number]).columns
    df_USGS[numeric_cols] = df_USGS[numeric_cols].interpolate(method='linear')

    # Variables for plotting
    variables_subplot1 = {
        "mag": {"color": "blue", "linestyle": "-", "alpha": 0.5},
        "nst": {"color": "orange", "linestyle": "--", "alpha": 0.5},
        "dmin": {"color": "green", "linestyle": "-.", "alpha": 0.5},
    }
    variables_subplot2 = {
        "rms": {"color": "red", "linestyle": "-", "alpha": 0.5},
        "depth": {"color": "blue", "linestyle": "--", "alpha": 0.5},
        "gap": {"color": "grey", "linestyle": "-.", "alpha": 0.5},
    }

    # Create subplots
    fig, axes = plt.subplots(2, 1, figsize=(15, 10), sharex=True)

    # Function to plot line graphs with independent y-axes and filled regions
    def plot_filled_lines(ax, x, y, label, color, linestyle, alpha):
        line, = ax.plot(x, y, color=color, linestyle=linestyle, label=label)
        ax.fill_between(x, 0, y, color=color, alpha=alpha)
        ax.set_ylabel(label, color=color)
        ax.tick_params(axis='y', labelcolor=color)
        return line

    # Plot Subplot 1
    offset = 0
    for var, props in variables_subplot1.items():
        ax = axes[0].twinx() if offset > 0 else axes[0] 
        plot_filled_lines(ax, df_USGS['Date'], df_USGS[var], var, 
                          props["color"], props["linestyle"], props["alpha"])
        if offset > 0:  # Adjust position of spines for additional y-axes
            ax.spines["right"].set_position(("outward", 60 * offset))
        offset += 1  # Increment offset for each new variable

    axes[0].set_title("The magnitude for the event (Mag), Number of seismic stations (nst), Distance from the epicenter (dmin).")
    axes[0].grid(True, linestyle="--", linewidth=0.5)

    # Plot Subplot 2
    offset = 0
    for var, props in variables_subplot2.items():
        ax = axes[1].twinx() if offset > 0 else axes[1] 
        plot_filled_lines(ax, df_USGS['Date'], df_USGS[var], var, 
                          props["color"], props["linestyle"], props["alpha"])
        if offset > 0:  # Adjust position of spines for additional y-axes
            ax.spines["right"].set_position(("outward", 60 * offset))
        offset += 1  # Increment offset for each new variable

    axes[1].set_title("Time residual (rms), Depth of rupture (Depth), Gap between stations (Gap)")
    axes[1].grid(True, linestyle="--", linewidth=0.5)

    # Format x-axis for clarity
    for ax in axes:
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
        ax.xaxis.set_major_locator(mdates.AutoDateLocator())  # Auto-tick based on date range
        ax.tick_params(axis='x', rotation=45)

    # Shared x-axis label
    axes[1].set_xlabel("Date")

    # Tighten layout and show plot
    plt.tight_layout()
    plt.show()


plot_earthquake_data_split_filled_scaled_combined(df_USGS)

# 4.0 Exploring EM-DATA fully

In [ ]:
def create_earthquake_map(df_quakes):
    """
    Another interactive map of earthquakes but just for EM_DAT and exploring more detals such as Magnitude, Total Deaths,
    Total Damage (Adjusted), Number of Injured, and Disaster ID. 
    The colour scale represents earthquake magnitude.

    Parameters:
        df_quakes (DataFrame): A pandas DataFrame with columns 'Latitude', 'Longitude', 'Magnitude',
                               'Total Deaths', 'Total Damage, Adjusted', 'No. Injured', and 'DisNo.'.

    Returns:
        None: Saves the map as an HTML file ('earthquake_map_end.html').
    """
    # Extract data for mapping
    Lat = df_quakes['Latitude']
    Lon = df_quakes['Longitude']
    Mag = df_quakes['Magnitude']
    Deaths = df_quakes['Total Deaths']
    Injured = df_quakes['No. Injured']
    Insured_Damage = df_quakes['Insured Damage, Adjusted']
    DisNo = df_quakes['DisNo.']  # Year/ID

    # Create a map centered around the mean coordinates
    m = folium.Map(location=[Lat.mean(), Lon.mean()], zoom_start=5)

    # Define a color scale for the Magnitude using 'viridis'
    min_mag = Mag.min()
    max_mag = Mag.max()
    
    # Create a Normalize object to map the magnitude to a color
    norm = mcolors.Normalize(vmin=min_mag, vmax=max_mag)
    cmap = plt.cm.get_cmap('viridis_r')  # 'viridis' is one of the color maps in Matplotlib

    # Add earthquake data points to the map
    for lat, lon, mag, deaths, damage, injured, insured_damage, disno in zip(Lat, Lon, Mag, Deaths, Damage, Injured, Insured_Damage, DisNo):
        # Normalize the magnitude for color scaling
        norm_mag = norm(mag)  # Normalized magnitude value
        
        # Get the color based on the normalized magnitude
        color = mcolors.to_hex(cmap(norm_mag))  # Convert to hex color code
        
        # Handle NaN values in each column - display "0" in the tooltip if missing
        deaths_text = "0" if np.isnan(deaths) else int(deaths)
        damage_text = "0" if np.isnan(damage) else int(damage)
        injured_text = "0" if np.isnan(injured) else int(injured)
        insured_damage_text = "0" if np.isnan(insured_damage) else int(insured_damage)
        
        # Create tooltip content
        tooltip_content = f"""
        <strong>Disaster ID:</strong> {disno}<br>
        <strong>Magnitude:</strong> {mag}<br>
        <strong>Total Deaths:</strong> {deaths_text}<br>
        <strong>Number Injured:</strong> {injured_text}<br>
        <strong>Total Damage (Adjusted) US$):</strong> {damage_text}<br>
        """
        
        # Create a circle marker with the magnitude-based color scale
        folium.CircleMarker(
            location=[lat, lon],
            radius=mag,  # Circle size proportional to magnitude
            color=color,  # Apply color based on magnitude
            fill=True,
            fill_opacity=0.6,
            tooltip=folium.Tooltip(tooltip_content)  # Add detailed tooltip
        ).add_to(m)

    # Save the map as an HTML file
    m.save('earthquake_map_end.html')

# Example usage:
# Assuming df_quakes contains the appropriate columns:
create_earthquake_map(df_quakes)


## 4.1 Variables in EM-DAT over time

In [ ]:
def plot_earthquake_data_split_filled_scaled_combined(df_quakes):
    """
    This function creates two subplots with primary y-axes showing Total Deaths and Insured Damage.
    
    Parameters:
        df_quakes (DataFrame): DataFrame with 'DisNo.', 'Total Deaths', 
                               'No. Injured', 'No. Affected', 'CPI', 
                               'Insured Damage, Adjusted', 'Total Damage, Adjusted'.
    """

    # Extract the year from 'DisNo.'
    df_quakes['Year'] = df_quakes['DisNo.'].str[:4].astype(int)
    df_quakes['Date'] = pd.to_datetime(df_quakes['Year'].astype(str), format='%Y')

    # Fill missing values in numeric columns
    numeric_cols = df_quakes.select_dtypes(include=[np.number]).columns
    df_quakes[numeric_cols] = df_quakes[numeric_cols].interpolate(method='linear')

    # Variables for plotting
    variables_subplot1 = {
        "No. Affected": {"color": "grey", "linestyle": "-", "alpha": 0.5},
        "No. Injured": {"color": "orange", "linestyle": "-.", "alpha": 0.5},
    }
    variables_subplot2 = {
        "CPI": {"color": "orange", "linestyle": "-", "alpha": 0.5},
        # Total Damage is already handled as a single axis, no need to repeat here
    }

    # Create subplots
    fig, axes = plt.subplots(2, 1, figsize=(15, 10), sharex=True)

    # Function to plot line graphs with independent y-axes and filled regions
    def plot_filled_lines(ax, x, y, label, color, linestyle, alpha):
        line, = ax.plot(x, y, color=color, linestyle=linestyle, label=label)
        ax.fill_between(x, 0, y, color=color, alpha=alpha)
        ax.set_ylabel(label, color=color)
        ax.tick_params(axis='y', labelcolor=color)
        return line

    # Plot Subplot 1
    # Primary y-axis (Total Deaths)
    axes[0].plot(df_quakes['Date'], df_quakes['Total Deaths'], color="blue", linestyle="-", label="Total Deaths")
    axes[0].fill_between(df_quakes['Date'], 0, df_quakes['Total Deaths'], color="blue", alpha=0.5)
    axes[0].set_ylabel("Total Deaths", color="blue")
    axes[0].tick_params(axis='y', labelcolor="blue")

    # Secondary y-axes (No. Affected, No. Injured) with adjustments to avoid overlap
    offset = 50
    for var, props in variables_subplot1.items():
        ax = axes[0].twinx()  # Create independent y-axis
        plot_filled_lines(ax, df_quakes['Date'], df_quakes[var], var, 
                          props["color"], props["linestyle"], props["alpha"])
        ax.spines["right"].set_position(("outward", offset))  # Adjust the position to prevent overlap
        offset += 50  # Increment offset for the next y-axis

    axes[0].set_title("Earthquake Impacts: Deaths, Affected, and Injured")
    axes[0].grid(True, linestyle="--", linewidth=0.5)

    # Plot Subplot 2
    # Primary y-axis (Insured Damage, Adjusted)
    axes[1].plot(df_quakes['Date'], df_quakes['Insured Damage, Adjusted'], color="black", linestyle="-", label="Insured Damage, Adjusted")
    axes[1].fill_between(df_quakes['Date'], 0, df_quakes['Insured Damage, Adjusted'], color="black", alpha=0.5)
    axes[1].set_ylabel("Insured Damage (US$)", color="black")
    axes[1].tick_params(axis='y', labelcolor="black")

    # Secondary y-axes (CPI)
    for var, props in variables_subplot2.items():
        ax = axes[1].twinx()  # Create independent y-axis
        plot_filled_lines(ax, df_quakes['Date'], df_quakes[var], var, 
                          props["color"], props["linestyle"], props["alpha"])

    # Label Total Damage axis explicitly (only one axis for Total Damage)
    ax_total_damage = axes[1].twinx()  # Use the same independent y-axis for Total Damage
    ax_total_damage.spines["right"].set_position(("outward", 60))  # Move it a bit to the right for clarity
    ax_total_damage.plot(df_quakes['Date'], df_quakes['Total Damage, Adjusted'], color="green", linestyle="-.", label="Total Damage, Adjusted")
    ax_total_damage.fill_between(df_quakes['Date'], 0, df_quakes['Total Damage, Adjusted'], color="green", alpha=0.5)
    ax_total_damage.set_ylabel("Total Damage (US$)", color="green")
    ax_total_damage.tick_params(axis='y', labelcolor="green")

    axes[1].set_title("Economic Impacts: Insured Damage, CPI, and Total Damage")
    axes[1].grid(True, linestyle="--", linewidth=0.5)

    # Format x-axis for clarity
    for ax in axes:
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
        ax.xaxis.set_major_locator(mdates.YearLocator(5))  # Major ticks every 5 years

    # Shared x-axis label
    axes[1].set_xlabel("Year")

    # Tighten layout and show plot
    plt.tight_layout()
    plt.show()

# Example usage:
plot_earthquake_data_split_filled_scaled_combined(df_quakes)


In [ ]:
####### Creating a table of averages within new magnitude range #######

def assign_magnitude_range(magnitude):
    '''Here we defined the magnitude ranges'''
    if 5.0 <= magnitude < 6.0:
        return "5.0-5.9"
    elif 6.0 <= magnitude < 7.0:
        return "6.0-6.9"
    elif 7.0 <= magnitude < 8.0:
        return "7.0-7.9"
    else:
        return "8.0+"


# Apply the magnitude range assignment
df_quakes['Magnitude'] = df_quakes['Magnitude'].apply(assign_magnitude_range)

# Select the variables of interest, excluding 'Excess Loss' and 'Reinsured Loss' for now
variables_of_interest = [
    'Magnitude', 'Total Deaths', 'No. Injured', 
    'No. Affected', 'Reconstruction Costs, Adjusted', 'Insured Damage, Adjusted', 
    'Total Damage, Adjusted', 'CPI', 'Start Year'
]
df_quakes_selected = df_quakes[variables_of_interest]

# Group by magnitude range and calculate the median for each column
grouped_ranges = df_quakes_selected.groupby('Magnitude').median(numeric_only=True).reset_index()

# Step 1: Categorize the quakes into decades
df_quakes_selected['Decade'] = (df_quakes_selected['Start Year'] // 10) * 10

# Step 2: Count the number of events in each decade
event_counts_by_decade = df_quakes_selected.groupby('Decade').size()

# Step 3: Calculate the probability per decade
decade_probabilities = event_counts_by_decade / 10  # Each decade has 10 years

# Step 4: Calculate weighted probability of each magnitude based on decade probabilities
total_years = df_quakes_selected['Start Year'].nunique()  # Total number of unique years
weighted_probabilities = []

for _, row in grouped_ranges.iterrows():
    # Extract the years for this magnitude range
    magnitude_range = row['Magnitude']
    years_in_magnitude = df_quakes_selected[df_quakes_selected['Magnitude'] == magnitude_range]['Start Year']
    
    # Get the unique decades this magnitude range appears in
    unique_decades = years_in_magnitude // 10 * 10
    
    # Calculate the weighted average probability for this magnitude
    weighted_probability = 0
    total_weight = 0
    
    for decade in unique_decades:
        # The weight for each decade is the number of years this magnitude appears in that decade
        years_in_decade = years_in_magnitude[years_in_magnitude // 10 * 10 == decade].nunique()
        weight = years_in_decade / 10  # Normalize by number of years in the decade
        
        # Add the weighted probability
        weighted_probability += (decade_probabilities.get(decade, 0) * weight)
        total_weight += weight

    # Normalize the weighted probability
    if total_weight > 0:
        weighted_probabilities.append(weighted_probability / total_weight)
    else:
        weighted_probabilities.append(0)

# Step 5: Add the calculated probabilities to the grouped results
grouped_ranges['Probability'] = weighted_probabilities

print(grouped_ranges)


## 4.2 Re-isurance metrics

In [ ]:
def calculate_weighted_probabilities(df_quakes, time_column='Start Year', magnitude_column='Magnitude'):
    """
    This function calculates a probability distribution for each magnitude based on weighted frequency 
    considering the irregular time series data (time gaps).
    """
    # Sort the dataframe by the time column to ensure events are in order
    df_quakes_sorted = df_quakes.sort_values(by=time_column)
    
    # Calculate the time differences between each earthquake event
    df_quakes_sorted['Time Gap'] = df_quakes_sorted[time_column].diff().fillna(0)
    
    # Calculate the weighted frequency for each magnitude
    # The weight is inversely proportional to the time gap: smaller gaps (more frequent events) get higher weights
    df_quakes_sorted['Weight'] = 1 / (df_quakes_sorted['Time Gap'] + 1e-6)  # Adding a small epsilon to avoid division by zero
    
    # Group by magnitude and sum the weights to get the weighted frequency
    weighted_frequencies = df_quakes_sorted.groupby(magnitude_column)['Weight'].sum().reset_index()
    
    # Normalize the weighted frequencies to get probabilities
    total_weight = weighted_frequencies['Weight'].sum()
    weighted_frequencies['Probability'] = weighted_frequencies['Weight'] / total_weight
    
    return weighted_frequencies[['Magnitude', 'Probability']]

def calculate_reinsurance_coverage_with_weighted_probabilities(df_quakes, premium_rate=0.10, attachment_point=50000000):
    """
    This function calculates various reinsurance-related metrics based on earthquake data:
    - Expected Loss
    - Excess Loss
    - Reinsurance Premium
    - Net Loss After Reinsurance
    - Final Adjusted Loss
    The probabilities are calculated using the weighted frequency of magnitudes in the dataset.
    """
    # Ensure the necessary columns exist
    required_columns = ['Magnitude', 'Insured Damage, Adjusted', 'Start Year']
    if not all(col in df_quakes.columns for col in required_columns):
        raise ValueError("DataFrame must contain 'Magnitude', 'Insured Damage, Adjusted', and 'Start Year' columns.")

    # Step 1: Calculate weighted probabilities based on the magnitudes
    weighted_probabilities = calculate_weighted_probabilities(df_quakes)

    # Step 2: Merge the weighted probabilities back into df_quakes
    df_quakes = df_quakes.merge(weighted_probabilities, on='Magnitude', how='left')

    # Step 3: Calculate Expected Loss (Insured Damage * Probability of Occurrence), accounting for '000 US $' units
    df_quakes['Expected Loss'] = (df_quakes['Insured Damage, Adjusted']) * df_quakes['Probability']

    # Step 4: Calculate Excess Loss (If Total Loss Exceeds the Attachment Point)
    df_quakes['Excess Loss'] = (df_quakes['Insured Damage, Adjusted']) - attachment_point
    df_quakes['Excess Loss'] = df_quakes['Excess Loss'].apply(lambda x: max(0, x))  # Only apply excess if > 0

    # Step 5: Calculate Reinsurance Premium (Expected Loss * Premium Rate)
    df_quakes['Reinsurance Premium'] = df_quakes['Expected Loss'] * premium_rate

    # Step 6: Calculate Net Loss After Reinsurance (Total Insured Damage - Excess Loss Covered by Reinsurer)
    df_quakes['Net Loss After Reinsurance'] = (df_quakes['Insured Damage, Adjusted']) - df_quakes['Excess Loss']

    # Step 7: Calculate Final Adjusted Loss (Net Loss After Reinsurance - Reinsurance Premium)
    df_quakes['Final Adjusted Loss'] = df_quakes['Net Loss After Reinsurance'] - df_quakes['Reinsurance Premium']

    # Return the updated DataFrame with all the calculated columns
    return df_quakes[['Magnitude', 'Insured Damage, Adjusted', 'Probability', 'Expected Loss', 
                      'Excess Loss', 'Reinsurance Premium', 'Net Loss After Reinsurance', 
                      'Final Adjusted Loss']]

df_quakes_updated = calculate_reinsurance_coverage_with_weighted_probabilities(df_quakes)


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

fig, axes = plt.subplots(2, 2, figsize=(18, 12))
axes = axes.flatten()

# Features to plot
features = ['Expected Loss', 'Excess Loss', 'Final Adjusted Loss', 'Reinsurance Premium']

# Plot each feature
for i, feature in enumerate(features):
    scatter = sns.scatterplot(
        data=df_quakes_updated,
        x='Magnitude',
        y=feature,
        size='Insured Damage, Adjusted',
        sizes=(20, 200),
        hue='Insured Damage, Adjusted',
        palette='viridis',
        ax=axes[i]
    )
    axes[i].set_title(f"{feature}", fontsize=18)
    axes[i].set_ylabel("(US$)", fontsize=18)  # Add 'US$' to y-axis label
    axes[i].set_xlabel("Magnitude", fontsize=18)  # Add label for x-axis and increase font size
    axes[i].grid(True)
    # Remove individual legends
    scatter.legend_.remove()

    # Increase tick size for both axes
    axes[i].tick_params(axis='both', which='major', labelsize=18)  # Adjust the size for both axes

# Adjust layout
plt.tight_layout()

# Add a shared color bar horizontally
cbar_ax = fig.add_axes([0.1, -0.05, 0.75, 0.03])  # Lowered the color bar
sm = plt.cm.ScalarMappable(cmap='viridis', norm=plt.Normalize(
    vmin=df_quakes_updated['Insured Damage, Adjusted'].min(),
    vmax=df_quakes_updated['Insured Damage, Adjusted'].max()
))
sm.set_array([])
cbar = plt.colorbar(sm, cax=cbar_ax, orientation='horizontal')
cbar.set_label('Adjusted Insured Damage (US$)', fontsize=18)

# Increase tick size for the color bar values
cbar.ax.tick_params(labelsize=16)  # Increase the color bar tick values' font size

# Flip x-axis for all subplots
for ax in axes:
    ax.invert_xaxis()

plt.show()


# 5.0 Comparison with other variables

In [ ]:
def plot_disaster_damage_heatmap(df):
    """
    This function generates six heatmaps for each diaster type in EM-DAT focusing on the Conterminous United States.:
    1. Total Damage
    2. Insured Damage
    3. CPI
    4. No. Affected
    5. No. Injured
    6. Total Deaths

    Parameters:
        df (DataFrame): DataFrame containing disaster data with 'Disaster Type', 'Country', 
                         'Latitude', 'Longitude', 'Total Damage, Adjusted', 
                         'Insured Damage, Adjusted', 'CPI', 'No. Affected',
                         'No. Injured', 'Total Deaths', and 'Start Year'.
    """
    # Define the boundaries for the Conterminous USA
    lat_min, lat_max = 24.6, 50.0
    lon_min, lon_max = -125.0, -65

    # Filter the data for the Conterminous USA
    df_conus = df[(df['Latitude'] >= lat_min) & 
                  (df['Latitude'] <= lat_max) & 
                  (df['Longitude'] >= lon_min) & 
                  (df['Longitude'] <= lon_max)].copy()

    # Remove the specific earthquake in Mexico
    df_conus = df_conus[df_conus['DisNo.'] != '1915-0004-USA']

    # Ensure 'Start Year' column exists and use it to create 'Year'
    if 'Start Year' in df_conus.columns:
        df_conus['Year'] = df_conus['Start Year']
    else:
        raise ValueError("Column 'Start Year' not found in the DataFrame.")
    
    # Get a list of all unique disaster types
    disaster_types = df_conus['Disaster Type'].unique()

    # Prepare a DataFrame to hold the aggregated data
    heatmap_data = pd.DataFrame()

    # Loop through each disaster type to gather data
    for disaster in disaster_types:
        df_disaster = df_conus[df_conus['Disaster Type'] == disaster]

        # Group by year and sum variables
        df_disaster_damage = df_disaster.groupby('Year')[
            ['Total Damage, Adjusted', 'Insured Damage, Adjusted', 'No. Affected', 'No. Injured', 'Total Deaths']
        ].sum()

        # Scale values for Total Damage and Insured Damage (adjusting by 1000 due to reloading data in)
        df_disaster_damage['Total Damage, Adjusted'] *= 1000 
        df_disaster_damage['Insured Damage, Adjusted'] *= 1000

        # If CPI exists, get the mean CPI per year per disaster type
        if 'CPI' in df_disaster.columns:
            df_disaster_damage['CPI'] = df_disaster.groupby('Year')['CPI'].mean()

        # Add the disaster type to the data
        df_disaster_damage['Disaster Type'] = disaster
        df_disaster_damage = df_disaster_damage.reset_index()  # Ensure 'Year' remains in the DataFrame
        heatmap_data = pd.concat([heatmap_data, df_disaster_damage], axis=0)


    heatmaps = {
        "Total Damage (Adjusted)": heatmap_data.pivot(index='Disaster Type', columns='Year', values='Total Damage, Adjusted'),
        "Insured Damage (Adjusted)": heatmap_data.pivot(index='Disaster Type', columns='Year', values='Insured Damage, Adjusted'),
        "CPI": heatmap_data.pivot(index='Disaster Type', columns='Year', values='CPI'),
        "No. Affected": heatmap_data.pivot(index='Disaster Type', columns='Year', values='No. Affected'),
        "No. Injured": heatmap_data.pivot(index='Disaster Type', columns='Year', values='No. Injured'),
        "Total Deaths": heatmap_data.pivot(index='Disaster Type', columns='Year', values='Total Deaths')
    }

    damage_vmin, damage_vmax = 0, 3e10
    affected_vmin, affected_vmax = 0, heatmap_data['No. Affected'].max()
    injured_vmin, injured_vmax = 0, heatmap_data['No. Injured'].max()
    deaths_vmin, deaths_vmax = 0, heatmap_data['Total Deaths'].max()

    fig, axes = plt.subplots(2, 3, figsize=(35, 22))  # Two rows for six heatmaps

    for ax, (title, data) in zip(axes.flatten(), heatmaps.items()):
        if "Total Damage" in title or "Insured Damage" in title:
            vmin, vmax = damage_vmin, damage_vmax
        elif "No. Affected" in title:
            vmin, vmax = affected_vmin, affected_vmax
        elif "No. Injured" in title:
            vmin, vmax = injured_vmin, injured_vmax
        elif "Total Deaths" in title:
            vmin, vmax = deaths_vmin, deaths_vmax
        else:
            vmin, vmax = None, None  # Default for CPI

        sns.heatmap(data, annot=False, cmap="viridis", fmt=",.0f" if vmin is not None else ",.2f", linewidths=.5,
                    cbar_kws={'label': 'Count' if 'No.' in title else 'US$ (Adjusted)' if 'Damage' in title else 'CPI'},
                    ax=ax, vmin=vmin, vmax=vmax)
        ax.set_title(title, fontsize=18)
        ax.set_xlabel('Year', fontsize=18)
        ax.set_ylabel('Disaster Type', fontsize=18)
        ax.tick_params(axis='x', rotation=90, labelsize=18)
        ax.tick_params(axis='y', labelsize=18)


        cbar = ax.collections[0].colorbar
        cbar.ax.tick_params(labelsize=18)
        cbar.set_label('Count' if 'No.' in title else 'US$ (Adjusted)' if 'Damage' in title else 'CPI', fontsize=20)  # Label font size


    plt.tight_layout()
    plt.show()

plot_disaster_damage_heatmap(df_Geo)


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

def plot_violinplots(df, filename="violin_plots.png"):
    """
    This function generates horizontal violin plots for each relevant column, grouped by disaster type.
    It shows min, median, max, and the distribution for each disaster type.
    """
    # List of columns to plot with their respective units
    cols = ['Total_Damage_Adjusted', 'Insured_Damage_Adjusted', 'No_Affected', 'No_Injured', 'Total_Deaths', 
            'CPI', 'Final_Adjusted_Loss', 'Expected_Loss', 'Excess_Loss']
    
    # Dictionary to hold units for each column
    units = {
        'Total_Damage_Adjusted': 'US$',
        'Insured_Damage_Adjusted': 'US$',
        'No_Affected': 'count',
        'No_Injured': 'count',
        'Total_Deaths': 'count',
        'CPI': 'Index',
        'Final_Adjusted_Loss': 'US$',
        'Expected_Loss': 'US$',
        'Excess_Loss': 'US$'
    }
    
    # Create a figure with subplots
    fig, axes = plt.subplots(3, 3, figsize=(25, 18))  # 3x3 grid for 9 metrics
    axes = axes.flatten()

    # Font sizes
    title_fontsize = 18
    label_fontsize = 18
    tick_fontsize = 16
    
    for i, col in enumerate(cols):
        # Create horizontal violin plot for each column grouped by 'Disaster Type'
        sns.violinplot(y='Disaster Type', x=col, data=df, ax=axes[i], palette="muted")
        title = col.replace('_', ' ')
        xlabel = f'{title} ({units[col]})'
        
        axes[i].set_title(f'{title}', fontsize=title_fontsize)
        axes[i].set_ylabel('Disaster Type', fontsize=label_fontsize)
        axes[i].set_xlabel(xlabel, fontsize=label_fontsize)
        
        axes[i].tick_params(axis='x', labelsize=tick_fontsize)
        axes[i].tick_params(axis='y', labelsize=tick_fontsize)
    
    plt.tight_layout()
    plt.savefig(filename, bbox_inches='tight', dpi=300)
    plt.show()

plot_violinplots(T, filename="/Users/isabelsmith/Documents/axa/violin_plots.png")


In [ ]:
def summary_table_by_disaster_type(df):
    """
    Returns a table showing median for each numeric column in the DataFrame,
    grouped by Disaster Type.
    """
    cols = ['Total_Damage_Adjusted', 'Insured_Damage_Adjusted', 'No_Affected', 'No_Injured', 'Total_Deaths', 
            'CPI', 'Final_Adjusted_Loss', 'Expected_Loss', 'Excess_Loss']

    summary = df.groupby('Disaster Type')[cols].agg([ 'median'])

    summary.columns = [f'{col} ({stat})' for col, stat in summary.columns]

    summary.index.name = 'Disaster Type'
    summary.reset_index(inplace=True)
    
    return summary